In [1]:
import pandas as pd
from datasets import load_from_disk, load_dataset
import numpy as np

import json
from fuzzywuzzy import process

c:\Users\chobi\anaconda3\envs\thesis\lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')


In [2]:
IMG_DIR = "https://cs.stanford.edu/people/rak248/"
"""
formart bouding box: [x_min,y_min,x_max,y_max]
"""

'\nformart bouding box: [x_min,y_min,x_max,y_max]\n'

In [3]:
ds = load_dataset("visual_genome", "region_descriptions_v1.0.0")

ds = ds['train']
df = ds.to_pandas()

Found cached dataset visual_genome (C:/Users/chobi/.cache/huggingface/datasets/visual_genome/region_descriptions_v1.0.0/1.0.0/6a4c7ae0afc696f22ba83bf725357bff6d74714cf3d24d4f07f5aed78d56b626)


  0%|          | 0/1 [00:00<?, ?it/s]

In [4]:
df.iloc[0].regions

array([{'region_id': 1382, 'image_id': 1, 'phrase': 'the clock is green in colour', 'x': 421, 'y': 57, 'width': 82, 'height': 139},
       {'region_id': 1383, 'image_id': 1, 'phrase': 'shade is along the street ', 'x': 194, 'y': 372, 'width': 182, 'height': 109},
       {'region_id': 1384, 'image_id': 1, 'phrase': 'man is wearing sneakers', 'x': 241, 'y': 491, 'width': 61, 'height': 30},
       {'region_id': 1385, 'image_id': 1, 'phrase': 'cars headlights are off', 'x': 617, 'y': 377, 'width': 36, 'height': 36},
       {'region_id': 1386, 'image_id': 1, 'phrase': 'bikes are parked at the far edge', 'x': 322, 'y': 298, 'width': 41, 'height': 49},
       {'region_id': 1432, 'image_id': 1, 'phrase': 'A sign on the facade of the building', 'x': 122, 'y': 14, 'width': 84, 'height': 178},
       {'region_id': 1433, 'image_id': 1, 'phrase': 'A tree trunk on the sidewalk', 'x': 624, 'y': 236, 'width': 96, 'height': 332},
       {'region_id': 1434, 'image_id': 1, 'phrase': 'A man in a red shirt

In [5]:
### normalize data (lowercase, remove punctional)
def norm_data(name):
    changed = []
    for n in name:
        rm_punctualtion =  n.strip().rstrip('.`, ')
        norm = rm_punctualtion.lower()
        changed.append(norm)

    f_series = pd.Series(changed)
    return f_series
    

In [6]:
def extract_data_from_original(ds):
    image_id = []
    image_url = []
    region_id = []
    phrases = []

    for i in range(len(ds)):
        for reg in ds.iloc[i].regions:
            image_id.append( ds.iloc[i].image_id)
            url = ds.iloc[i].url
            url = url.split("https://cs.stanford.edu/people/rak248/")[1]
            image_url.append(url)
            region_id.append(reg.get('region_id'))
            phrases.append(reg.get('phrase'))

            
    dict = {'image_url':image_url, 'image_url':image_url, 'region_id': region_id, 'phrases': phrases}
    data = pd.DataFrame(dict)
    return data

In [7]:
extracted = extract_data_from_original(df)

In [8]:
extracted['phrases'] = norm_data(extracted['phrases'])

In [9]:
extracted

,image_url,region_id,phrases
0,VG_100K_2/1.jpg,1382,the clock is green in colour
1,VG_100K_2/1.jpg,1383,shade is along the street
2,VG_100K_2/1.jpg,1384,man is wearing sneakers
3,VG_100K_2/1.jpg,1385,cars headlights are off
4,VG_100K_2/1.jpg,1386,bikes are parked at the far edge
...,...,...,...
4297497,VG_100K_2/2417997.jpg,4931341,a crystal clear blue sky
4297498,VG_100K_2/2417997.jpg,4931342,a green bus on road
4297499,VG_100K_2/2417997.jpg,4931343,a green bus on street
4297500,VG_100K_2/2417997.jpg,4931344,a bus with doors open


In [10]:
objects = []
with open('dataset/objects_vocab.txt', 'r') as f:
    for line in f:
        objects.append(np.array(line.strip().split(',')))

In [11]:
objects

[array(['yolk'], dtype='<U4'),
 array(['goal'], dtype='<U4'),
 array(['bathroom'], dtype='<U8'),
 array(['macaroni'], dtype='<U8'),
 array(['umpire'], dtype='<U6'),
 array(['toothpick'], dtype='<U9'),
 array(['alarm clock'], dtype='<U11'),
 array(['ceiling fan'], dtype='<U11'),
 array(['photos'], dtype='<U6'),
 array(['parrot'], dtype='<U6'),
 array(['tail fin'], dtype='<U8'),
 array(['birthday cake'], dtype='<U13'),
 array(['calculator'], dtype='<U10'),
 array(['catcher'], dtype='<U7'),
 array(['toilet'], dtype='<U6'),
 array(['batter'], dtype='<U6'),
 array(['stop sign', 'stopsign'], dtype='<U9'),
 array(['cone'], dtype='<U4'),
 array(['microwave', 'microwave oven'], dtype='<U14'),
 array(['skateboard ramp'], dtype='<U15'),
 array(['tea'], dtype='<U3'),
 array(['dugout'], dtype='<U6'),
 array(['products'], dtype='<U8'),
 array(['halter'], dtype='<U6'),
 array(['kettle'], dtype='<U6'),
 array(['kitchen'], dtype='<U7'),
 array(['refrigerator', 'fridge'], dtype='<U12'),
 array(['ostrich

In [21]:
def search_objectsnum(phrase):
    phrase = np.array(phrase.split())
    for i, word in enumerate(phrase):
        match = process.extractOne(word,objects, score_cutoff=100)
        if match is not None:
            print(match[0])
            return True
        else:
            return False

In [13]:
extracted

,image_url,region_id,phrases
0,VG_100K_2/1.jpg,1382,the clock is green in colour
1,VG_100K_2/1.jpg,1383,shade is along the street
2,VG_100K_2/1.jpg,1384,man is wearing sneakers
3,VG_100K_2/1.jpg,1385,cars headlights are off
4,VG_100K_2/1.jpg,1386,bikes are parked at the far edge
...,...,...,...
4297497,VG_100K_2/2417997.jpg,4931341,a crystal clear blue sky
4297498,VG_100K_2/2417997.jpg,4931342,a green bus on road
4297499,VG_100K_2/2417997.jpg,4931343,a green bus on street
4297500,VG_100K_2/2417997.jpg,4931344,a bus with doors open


In [18]:
filter = extracted.sample(20000)

In [19]:
filter

,image_url,region_id,phrases
3777015,VG_100K/2326743.jpg,4255352,blue jeans on a man
2688335,VG_100K/2357309.jpg,2796655,blue shorts on the boy
4071398,VG_100K/2316345.jpg,4553084,green lettuce on turkey sandwich
3219028,VG_100K/2345937.jpg,3339464,person wearing white v-neck shirt
34941,VG_100K_2/867.jpg,5526678,white man eating an orange
...,...,...,...
2354511,VG_100K/2364478.jpg,2453679,short green and brown grass
3844572,VG_100K/2323244.jpg,4622503,big ship in the clear water
53816,VG_100K_2/1529.jpg,5540267,white man eating an orange
1537879,VG_100K_2/2382331.jpg,1406216,the man's shirt is checkered


In [22]:

extracted_filter = filter[filter['phrases'].apply(lambda x:search_objectsnum(x))]



['person']
['bear']
['man']
['boy']
['drain']
['front']
['nose']
['lights']
['woman']
['boy']
['black']
['light']
['railing']
['person']
['kid']
['neck']
['pile']
['ramp']
['bike']
['wheel']
['boat']
['hole']
['man']
['table']
['man']
['pencil']
['boy']
['yellow']
['lights']
['pedal']
['person']
['man']
['food']
['woman']
['tracks']
['shoes']
['horns']
['sky']
['boys']
['leaves']
['line']
['people']
['gate']
['man']
['shutters']
['baseball']
['teddy']
['person']
['black']
['dog']
['windows']
['restroom']
['man']
['oranges']
['black']
['black']
['yellow']
['watch']
['shoe']
['whiskers']
['rocks']
['piece']
['nose']
['black']
['hands']
['bananas']
['black']
['black']
['pocket']
['black']
['guy']
['window']
['black']
['wall']
['people']
['tie']
['hand']
['bags']
['shadow']
['planes']
['man']
['balcony']
['eye']
['toilet']
['back']
['olive']
['group']
['yellow']
['teddy']
['black']
['cars']
['people']
['woman']
['flowers']
['leg']
['cows']
['chicken']
['vehicle']
['ears']
['black']
['light

['black']
['lady']
['knife']
['girl']
['bear']
['grass']
['top']
['bun']
['burners']
['black']
['man']
['reflection']
['photo']
['black']
['top']
['snow']
['wine']
['horses']
['people']
['sky']
['clock']
['black']
['picture']
['fence']
['light']
['bike']
['person']
['black']
['person']
['picture']
['neck']
['label']
['light']
['head']
['child']
['sign']
['dog']
['clouds']
['sticker']
['black']
['remote']
['person']
['speaker']
['birds']
['dinner']
['trees']
['wrist']
['panda']
['person']
['cross']
['poles']
['bench']
['lights']
['black']
['people']
['dirt']
['mane']
['avocado']
['people']
['man']
['water']
['wheel']
['skier']
['front']
['handles']
['glass']
['snow']
['front']
['lights']
['street']
['knee']
['yellow']
['horse']
['plant']
['woman']
['corn']
['man']
['window']
['animal']
['black']
['clothes']
['laptop']
['black']
['sheep']
['man']
['cabinet']
['umpire']
['black']
['window']
['trees']
['black']
['people']
['earring']
['arrow']
['zebra']
['patch']
['man']
['yellow']
['numbe

KeyboardInterrupt: 